In [1]:
# pip install pandas numpy tensorflow scikit-learn matplotlib tqdm


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical



In [3]:
# Load the dataset
file_path = '/Users/pvuda/Documents/features/combined_file.csv'
data = pd.read_csv(file_path)

In [15]:
# Display the first few rows of the dataframe
display(data.head())
print("Features:", data.columns.to_list())
print("Class of real_or_fake:", set(data['real_or_fake']))

,audio_id,real_or_fake,spectral_centroid_mean,spectral_centroid_std,spectral_centroid_var,spectral_centroid_min,spectral_centroid_max,spectral_centroid_range,spectral_centroid_25th_percentile,spectral_centroid_50th_percentile,...,shimmer_dda,hnr,voicedcount,npause,originaldur,intensity_duration,speakingrate,articulationrate,asd,totalpauseduration
0,p241_171,1,1264.074015,822.964906,677271.236891,422.119966,4438.950839,4016.830872,750.791935,1028.198334,...,0.118602,10.488762,13.0,0.0,4.140000,4.140000,3.140097,5.416667,0.184615,1.740000
1,p241_171,5,1092.732243,835.096515,697386.189558,299.390288,4276.826478,3977.436190,569.495673,859.165036,...,0.120658,11.283641,14.0,0.0,4.160000,4.160000,3.365385,5.794702,0.172571,1.744000
2,p241_171,0,1239.497052,842.612526,709995.868275,288.564361,4151.701698,3863.137337,764.123650,1024.689694,...,0.125836,10.476980,14.0,0.0,3.474125,3.474125,4.029792,5.570892,0.179504,0.961063
3,p241_171,4,1218.714047,851.013722,724224.354809,384.025330,4499.744355,4115.719024,723.722157,988.215478,...,0.136116,9.717817,12.0,0.0,4.143750,4.143750,2.895928,5.000000,0.200000,1.743750
4,p241_171,3,1090.133359,872.704547,761613.225871,206.065685,4304.859443,4098.793758,531.764632,867.988671,...,0.186246,8.789596,14.0,0.0,4.146667,4.146667,3.376206,5.756579,0.173714,1.714667


Features: ['audio_id', 'real_or_fake', 'spectral_centroid_mean', 'spectral_centroid_std', 'spectral_centroid_var', 'spectral_centroid_min', 'spectral_centroid_max', 'spectral_centroid_range', 'spectral_centroid_25th_percentile', 'spectral_centroid_50th_percentile', 'spectral_centroid_75th_percentile', 'spectral_centroid_skew', 'spectral_centroid_kurtosis', 'spectral_bandwidth_mean', 'spectral_bandwidth_std', 'spectral_bandwidth_var', 'spectral_bandwidth_min', 'spectral_bandwidth_max', 'spectral_bandwidth_range', 'spectral_bandwidth_25th_percentile', 'spectral_bandwidth_50th_percentile', 'spectral_bandwidth_75th_percentile', 'spectral_bandwidth_skew', 'spectral_bandwidth_kurtosis', 'spectral_contrast_mean', 'spectral_contrast_std', 'spectral_contrast_var', 'spectral_contrast_min', 'spectral_contrast_max', 'spectral_contrast_range', 'spectral_contrast_25th_percentile', 'spectral_contrast_50th_percentile', 'spectral_contrast_75th_percentile', 'spectral_contrast_skew', 'spectral_contrast_k

In [16]:
# Convert 'real_or_fake' to numerical labels
label_encoder = LabelEncoder()
data['real_or_fake'] = label_encoder.fit_transform(data['real_or_fake'])

In [17]:
# Define features and target
X = data.drop(columns=['audio_id', 'real_or_fake']).values
y = data['real_or_fake'].values

# Convert labels to one-hot encoding
y = to_categorical(y)

In [21]:
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
# Define the RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 408, 128)       │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 408, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 14)             │           910 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 116,878 (456.55 KB)

 Trainable params: 116,878 (456.55 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
# Reshape the data to fit the LSTM layer's expected input shape
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=2, batch_size=128, validation_data=(X_test_reshaped, y_test))

Epoch 1/5
1697/1697 ━━━━━━━━━━━━━━━━━━━━ 2255s 1s/step - accuracy: 0.1288 - loss: nan - val_accuracy: 0.1283 - val_loss: nan
Epoch 2/5
1496/1697 ━━━━━━━━━━━━━━━━━━━━ 4:18 1s/step - accuracy: 0.1278 - loss: nan

KeyboardInterrupt: 

In [28]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical

# Load and preprocess the data
file_path = '/Users/pvuda/Documents/features/combined_file.csv'
data = pd.read_csv(file_path)

label_encoder = LabelEncoder()
data['real_or_fake'] = label_encoder.fit_transform(data['real_or_fake'])

X = data.drop(columns=['audio_id', 'real_or_fake'])
y = data['real_or_fake']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Sample 30% of the data if needed
data_sampled = data.sample(frac=0.3, random_state=42)
X_scaled = scaler.fit_transform(data_sampled.drop(columns=['audio_id', 'real_or_fake']))
y = data_sampled['real_or_fake']

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Reshape the data for LSTM
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# One-hot encode the labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the RNN model
model = Sequential()
model.add(LSTM(128, input_shape=(X_train_reshaped.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(len(np.unique(y)), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
history = model.fit(X_train_reshaped, y_train, validation_data=(X_test_reshaped, y_test), epochs=20, batch_size=32, verbose=1)

# Plot accuracy and loss if needed
# import matplotlib.pyplot as plt
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('Model accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend(['Train', 'Validation'], loc='upper left')
# plt.show()


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 408, 128)       │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 408, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 14)             │           910 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 116,878 (456.55 KB)

 Trainable params: 116,878 (456.55 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 860s 422ms/step - accuracy: 0.1316 - loss: nan - val_accuracy: 0.1265 - val_loss: nan
Epoch 2/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 751s 369ms/step - accuracy: 0.1285 - loss: nan - val_accuracy: 0.1265 - val_loss: nan
Epoch 3/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 695s 341ms/step - accuracy: 0.1307 - loss: nan - val_accuracy: 0.1265 - val_loss: nan
Epoch 4/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 813s 399ms/step - accuracy: 0.1274 - loss: nan - val_accuracy: 0.1265 - val_loss: nan
Epoch 5/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 826s 405ms/step - accuracy: 0.1275 - loss: nan - val_accuracy: 0.1265 - val_loss: nan
Epoch 6/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 765s 376ms/step - accuracy: 0.1281 - loss: nan - val_accuracy: 0.1265 - val_loss: nan
Epoch 7/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 684s 336ms/step - accuracy: 0.1275 - loss: nan - val_accuracy: 0.1265 - val_loss: nan
Epoch 8/20
2037/2037 ━━━━━━━━━━━━━━━━━━━━ 742s 364ms/step - accuracy: 0.1273 - loss: nan - val_accuracy:

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

plt.show()

In [ ]:
import kerastuner as kt

def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32), input_shape=(X_train.shape[1], 1), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=hp.Int('units', min_value=32, max_value=256, step=32)))
    model.add(Dropout(0.2))
    model.add(Dense(y_train.shape[1], activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5, executions_per_trial=3, directory='tuning', project_name='audio_classification')

tuner.search(X_train_reshaped, y_train, epochs=50, validation_data=(X_test_reshaped, y_test))
best_model = tuner.get_best_models(num_models=1)[0]

best_model.summary()
